In [2]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import tensorflow as tf

mnist = read_data_sets('MNIST_data/', one_hot=True)

/Users/lyon/.pyenv/versions/3.6.4/envs/scikit-learn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def weight_variable(shape):
    with tf.name_scope('weights'):
        initial = tf.truncated_normal(shape, stddev=0.1)
        weight = tf.Variable(initial, name='W')
        
    return weight

def bias_variable(shape):
    with tf.name_scope('biases'):
        initial = tf.constant(0.1, shape=shape)
        biases = tf.Variable(initial, name='b')
        
    return biases

def add_fc_layer(x, in_size, out_size, activation_function=None):
    with tf.name_scope('fc_layer'):
        Weights = weight_variable([in_size, out_size])
        biases = bias_variable([1, out_size])
            
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(x, Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    
    tf.summary.histogram('weights', Weights)
    tf.summary.histogram('biases', biases)
    
    return outputs

def add_conv_layer(x, kernel_shape, color_channel, kernel_num):
    with tf.name_scope('conv'):
        W_conv = weight_variable(kernel_shape+[color_channel, kernel_num])
        b_conv = bias_variable([kernel_num])
        
        h_conv = tf.nn.relu(tf.nn.conv2d(x, W_conv, strides=[1, 1, 1, 1], padding='SAME'))
        
        h_pool = tf.nn.max_pool(h_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        tf.summary.histogram('W_conv', W_conv)
        tf.summary.histogram('b_conv', b_conv)
    
    return h_pool

In [6]:
with tf.Graph().as_default() as g:
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, [None, 784])
        y_ = tf.placeholder(tf.float32, [None, 10])
        x_image = tf.reshape(x, [-1, 28, 28, 1])

    conv1 = add_conv_layer(x_image, [5, 5], 1, 32)

    with tf.name_scope('Image_output_conv1'):
        conv1_image = conv1[0:1, :, :, 0:32]
        conv1_image = tf.transpose(conv1_image, perm=[3,1,2,0])
        tf.summary.image('Image_output_conv1', conv1_image, max_outputs=32)
        
    conv2 = add_conv_layer(conv1, [5, 5], 32, 64)
    
    with tf.name_scope('Image_output_conv2'):
        conv2_image = conv2[0:1, :, :, 0:64]
        conv2_image = tf.transpose(conv2_image, perm=[3,1,2,0])
        tf.summary.image('Image_output_conv2', conv2_image, max_outputs=64)
    
    conv2_flat = tf.reshape(conv2, [-1, 7*7*64])
    h_fc1 = add_fc_layer(conv2_flat, 7*7*64, 1024, tf.nn.relu)
    
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        tf.summary.scalar('dropout_keep_probability', keep_prob)
    
    y_conv = add_fc_layer(h_fc1_drop, 1024, 10, tf.nn.softmax)
    
    # 损失函数
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))

    tf.summary.scalar('loss', cross_entropy)

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge_all()
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter('./logs/conv', sess.graph)

        tf.global_variables_initializer().run()

        for i in range(2000):
            batch = mnist.train.next_batch(50)

            if i%100 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
                print("step %d, training accuracy %g"%(i, train_accuracy))

            summary, _ = sess.run([merged, train_step], feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            writer.add_summary(summary, i)

step 0, training accuracy 0.06
step 100, training accuracy 0.42
step 200, training accuracy 0.52
step 300, training accuracy 0.62
step 400, training accuracy 0.66
step 500, training accuracy 0.8
step 600, training accuracy 0.84
step 700, training accuracy 0.7
step 800, training accuracy 0.9
step 900, training accuracy 0.82
step 1000, training accuracy 0.9
step 1100, training accuracy 0.78
step 1200, training accuracy 0.86
step 1300, training accuracy 0.96
step 1400, training accuracy 0.92
step 1500, training accuracy 0.9
step 1600, training accuracy 0.94
step 1700, training accuracy 0.84
step 1800, training accuracy 0.92
step 1900, training accuracy 0.86
